## Programming Environment

In [1]:
# 3rd Party
import tensorflow as tf
import numpy as np
from object_detection.metrics.coco_tools import COCOWrapper,COCOEvalWrapper
# Python STL
from typing import List,Tuple,Dict
import os
import pdb
import json

In [2]:
from preprocess.construct_dicts import construct_dicts, construct_desired_ids
from preprocess.preprocess import preprocess_train_images, preprocess_validation_images, preprocess_test_images, \
  construct_category_index, get_unsliced_images_np, calculate_label_id_offsets, map_category_ids_to_index
from retrain.retrain import retrain
from utils.load_data import get_annotations
from utils.plot import visualize_image_set
from evaluate.detect import run_inference
from evaluate.postprocess import restore_image_detections, run_nms
from evaluate.eval import write_window_validation_file, write_window_results, evaluate
from retrain.retrain import convert_to_tensors


In [3]:
from ssd.ssd512_vgg16 import SSD512_VGG16

In [4]:
log_dir = "./tb"

## Training Setup

In [5]:
import os
data_path = "./dota_sports_data"
# set image path info
train_image_dir = data_path + '/train/images/' 
train_annotations_dir = data_path + '/annotations/train.json'
# open annotation information
train_annotations = get_annotations(train_annotations_dir)
desired_categories = {'tennis-court','soccer-ball-field','ground-track-field','baseball-diamond'}
desired_ids = construct_desired_ids(desired_categories, train_annotations['categories'])
# construct dictionaries containing info about images
(train_images_dict, train_file_name_dict) = construct_dicts(train_annotations)
# create category index in the correct format for retraining and detection
category_index = construct_category_index(train_annotations, desired_categories)
label_id_offsets = calculate_label_id_offsets(category_index)

{2: {'id': 2, 'name': 'baseball-diamond'}, 4: {'id': 4, 'name': 'ground-track-field'}, 8: {'id': 8, 'name': 'tennis-court'}, 11: {'id': 11, 'name': 'soccer-ball-field'}}


In [6]:
# set windowing information (size of window and stride); these values taken from DOTA paper
win_height = 1024
win_width  = 1024
win_stride_vert  = 512
win_stride_horiz = 512
win_set = (win_height, win_width, win_stride_vert, win_stride_horiz) # windowing information

In [7]:
(train_images_np, gt_boxes, gt_classes) = preprocess_train_images(
  train_images_dict, train_file_name_dict, train_image_dir,train_annotations,
  category_index, win_set,verbose = False)

gt_classes = map_category_ids_to_index(label_id_offsets, gt_classes)
(train_image_tensors, gt_box_tensors, gt_classes_one_hot_tensors) = convert_to_tensors(
        train_images_np, gt_boxes, gt_classes, label_id_offsets, len(category_index))                                                     

In [8]:
valid_annotations_dir  = data_path + '/annotations/validation.json'

valid_image_dir  = data_path + '/validation/images/'
valid_annotations  = get_annotations(valid_annotations_dir)
(valid_images_dict, valid_file_name_dict)   = construct_dicts(valid_annotations)
(valid_images_np, valid_gt_boxes, valid_gt_classes, valid_images_dict, valid_no_annotation_ids) = preprocess_validation_images(
  valid_images_dict, valid_file_name_dict, valid_image_dir, valid_annotations,
  category_index, win_set, verbose=False)
(valid_image_tensors, valid_box_tensors, valid_class_tensors) = convert_to_tensors(
    valid_images_np, valid_gt_boxes, valid_gt_classes, label_id_offsets, len(category_index)
)

In [9]:
from collections import namedtuple
ObjDataset = namedtuple("ObjDataset", ["images", "boxes", "classes"])
train_dataset = ObjDataset(train_image_tensors, gt_box_tensors, gt_classes_one_hot_tensors)
valid_dataset = ObjDataset(
                            [t for (i, t) in enumerate(valid_image_tensors) if i not in valid_no_annotation_ids],
                            [t for (i, t) in enumerate(valid_box_tensors) if i not in valid_no_annotation_ids],
                            [t for (i, t) in enumerate(valid_class_tensors) if i not in valid_no_annotation_ids]
)

In [10]:
def coco_eval(data_path, annotations, images_np, images_dict, desired_ids, label_id_offsets, model):
  results_path = os.path.join(data_path, "annotations", "evaluation", "window_results.json")
  labels_path = os.path.join(data_path, "annotations", "validation_window.json")

  test_images_dict, predicted_boxes, predicted_classes, predicted_scores = run_inference(images_np, images_dict, label_id_offsets, model)
  write_window_validation_file(data_path, annotations, images_dict)
  write_window_results(results_path, images_dict, min_threshold=.01)

  with open(labels_path, "r") as r: cocoGt = COCOWrapper(json.load(r))
  cocoDt = cocoGt.loadRes(results_path)
  cocoEval = COCOEvalWrapper(cocoGt, cocoDt, iou_type="bbox", agnostic_mode=True)
  cocoEval.params.catIds = list(desired_ids) # set category ids we want to evaluate on
  metrics = cocoEval.ComputeMetrics()
  return metrics 

class COCOEvalCallback(tf.keras.callbacks.Callback):
  def __init__(self, coco_eval_kwargs, log_dir):
    super(COCOEvalCallback, self).__init__()
    self._coco_eval_kwargs = coco_eval_kwargs
    os.makedirs(log_dir, exist_ok=True)
    self._writer = tf.summary.create_file_writer(log_dir)
    self._metrics = [
                     "Precision/mAP",
                     "Precision/mAP@.50IOU",
                     "Precision/mAP@.75IOU",
                     "Recall/AR@100"
    ]

  def on_epoch_begin(self, epoch, logs = None):
    pass

  def on_epoch_end(self, epoch, logs = None):
    metrics_dict = coco_eval(**self._coco_eval_kwargs)
    with self._writer.as_default():
      for m in self._metrics:
        tf.summary.scalar(m, metrics_dict[0][m], step=epoch)
    self._writer.flush()

In [11]:
class SummaryScalarCallback(tf.keras.callbacks.Callback):
  def __init__(self, log_dir):
    super(SummaryScalarCallback, self).__init__()
    os.makedirs(log_dir, exist_ok=True)
    self._writer = tf.summary.create_file_writer(log_dir)
  def on_epoch_begin(self, epoch, logs = None):
    pass
  def on_epoch_end(self, epoch, logs = None):
    if logs is None: return
    with self._writer.as_default():
      for (k,v) in logs.items():
        tf.summary.scalar(k, v, step=epoch)
    self._writer.flush()


In [12]:
def save_checkpoint(ckpt: tf.train.Checkpoint, ckpt_path: str):
  dir_name = os.path.dirname(ckpt_path)
  os.makedirs(dir_name, exist_ok=True)
  ckpt.save(ckpt_path)

class ModelCheckpointCallback(tf.keras.callbacks.Callback):
  def __init__(self, ckpt: tf.train.Checkpoint, ckpt_path: str):
    super(ModelCheckpointCallback, self).__init__()
    self._ckpt = ckpt
    self._ckpt_path = ckpt_path
  def on_epoch_begin(self, epoch, logs = None):
    pass
  def on_epoch_end(self, epoch, logs = None):
    save_checkpoint(self._ckpt, self._ckpt_path)

In [20]:
import pdb

def shuffle_and_batch(dataset: tf.data.Dataset, batch_size=8, random_seed=0) -> tf.data.Dataset:
  dataset = dataset.shuffle(buffer_size=batch_size*4,seed=random_seed)
  dataset = dataset.batch(batch_size)
  return dataset


def get_valid_loss_fn(valid_dataset: ObjDataset, model, batch_size=8):
  if not valid_dataset: return lambda: 0.
  (image_tensors, box_tensors, class_tensors) = valid_dataset
  image_tensors = tf.squeeze( tf.stack([model.preprocess(t)[0] for t in image_tensors], axis=0) )
  raw_index_dataset = tf.data.Dataset.from_generator(lambda: range(len(image_tensors)), output_types=tf.int32)
  valid_loss = tf.keras.metrics.Mean("valid_loss", dtype=tf.float32)
  def f():
    index_dataset = raw_index_dataset.batch(batch_size)
    for indices in index_dataset:
      model.provide_groundtruth(
          [box_tensors[i] for i in indices],
          [class_tensors[i] for i in indices]
      )
      prediction_dict = model.predict(tf.gather(image_tensors, indices), None)
      loss_val = model.loss(prediction_dict)["WeightedTotal"]
      valid_loss(loss_val)
    result = valid_loss.result()
    valid_loss.reset_states()
    return result
  return f

def train_model(model, optimizer, train_dataset,
                batch_size : int = 8,
                num_epochs : int = 1,
                epoch_start : int = 1,
                valid_dataset = None,
                callbacks : List[tf.keras.callbacks.Callback] = []):


  to_fine_tune = [v for v in model.trainable_variables]
  valid_loss_fn = get_valid_loss_fn(valid_dataset, model, batch_size=batch_size)
  (image_tensors, gt_box_tensors, gt_classes_one_hot_tensors) = train_dataset

  image_tensors = tf.squeeze( tf.stack([model.preprocess(t)[0] for t in image_tensors], axis=0) )

  train_loc_loss = tf.keras.metrics.Mean("train_loc_loss", dtype=tf.float32)
  train_conf_loss = tf.keras.metrics.Mean("train_conf_loss", dtype=tf.float32)

  raw_dataset = tf.data.Dataset.from_generator(lambda: range(len(image_tensors)), output_types=tf.int32)
  for i_epoch in range(epoch_start, epoch_start + num_epochs):
    for c in callbacks: c.on_epoch_begin(i_epoch)
    dataset = shuffle_and_batch(raw_dataset, batch_size=batch_size, random_seed=i_epoch)
    for (i_batch, indices) in enumerate(dataset):
      for c in callbacks: c.on_train_batch_begin(i_batch)
      batch_images = tf.gather(image_tensors, indices)
      model.provide_groundtruth(
          [gt_box_tensors[i] for i in indices],
          [gt_classes_one_hot_tensors[i] for i in indices]
      )
      with tf.GradientTape() as tape:
        prediction_dict = model.predict(batch_images, None)
        loss_dict = model.loss(prediction_dict)
      gradients = tape.gradient(loss_dict["WeightedTotal"], to_fine_tune)
      optimizer.apply_gradients( zip(gradients, to_fine_tune) )
      train_loc_loss(loss_dict["Localization"])
      train_conf_loss(loss_dict["Confidence"])
      for c in callbacks: c.on_train_batch_end(i_batch, logs = {})
#       break

    losses_dict = {
        "Loss/Train/Localization": train_loc_loss.result(),
        "Loss/Train/Confidence": train_conf_loss.result(),
        "Loss/Validation": valid_loss_fn()
    }
    for c in callbacks: c.on_epoch_end(i_epoch, losses_dict)
    train_loc_loss.reset_states()
    train_conf_loss.reset_states()


In [14]:
checkpoints_root = "./checkpoints"
vars_checkpoint_dir = os.path.join(checkpoints_root, "ssd-variables")
opt_checkpoint_dir = os.path.join(checkpoints_root, "ssd-optimizer")
vars_checkpoint_path = os.path.join(vars_checkpoint_dir, "ckpt")
opt_checkpoint_path = os.path.join(opt_checkpoint_dir, "ckpt")

In [25]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9)
optimizer_checkpoint = tf.train.Checkpoint(optimizer=optimizer)

In [16]:
load_old_model = False
if load_old_model:
  model = SSD512_VGG16.from_checkpoint(len(category_index), tf.train.latest_checkpoint(vars_checkpoint_dir))
else:
  weights_path = "/data/pretrained_models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
  model = SSD512_VGG16.from_scratch(len(category_index), weights_path)

In [17]:
load_old_optimizer = False
if load_old_optimizer:
  optimizer_checkpoint.restore( tf.train.latest_checkpoint(opt_checkpoint_dir) )

In [30]:
import os
callbacks = []

coco_eval_callback = COCOEvalCallback({
    "model": model, "images_np": valid_images_np, "images_dict": valid_images_dict,
    "desired_ids": desired_ids, "label_id_offsets": label_id_offsets, "annotations": valid_annotations,
    "data_path": data_path
},log_dir=os.path.join(log_dir, "coco"))
callbacks.append(coco_eval_callback)


summary_callback = SummaryScalarCallback(log_dir=os.path.join(log_dir, "metrics"))
callbacks.append(summary_callback)

model_save_callback = ModelCheckpointCallback(model.checkpoint, vars_checkpoint_path)
callbacks.append(model_save_callback)

opt_save_callback = ModelCheckpointCallback(optimizer_checkpoint, opt_checkpoint_path)
callbacks.append(opt_save_callback)



In [ ]:
train_model(model,optimizer,train_dataset,
            valid_dataset=valid_dataset,
            batch_size=4, num_epochs=30, epoch_start=50, callbacks=callbacks)


creating index...
index created!
Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.42s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.019
 Average Recall     (AR) @[ IoU=0